# Digestão

Neste caderno usaremos as estações válidas encontradas para digerir as séries históricas e compilar um banco de dados com todos os registros que se enquadram nos nossos critérios.

In [1]:
import os

files = os.listdir('time_series')
files = [f for f in files if f.endswith('_Chuvas.csv')]
files.sort()

print(len(files))
print(files[:5])

359
['1036003_Chuvas.csv', '1036004_Chuvas.csv', '1036005_Chuvas.csv', '1036007_Chuvas.csv', '1036009_Chuvas.csv']


In [2]:
import pandas as pd

stations = pd.read_csv('stations.csv')

Aqui procuramos nos arquivos por entradas:
* Registros feitos por pluviômetros;
* O total mensal está presente;
* Entre 2004 e 2024, inclusivo.

In [ ]:
db = pd.DataFrame()

shortest_df = 0
longest_df = 0

for file in files:
    # load csv skipping 14 header lines
    df = pd.read_csv(
        f"time_series/{file}",
        encoding="utf-8",
        sep=";",
        skiprows=14,
        decimal=",",
        parse_dates=["Data"],
    )

    # get data were interested in
    df = df[["EstacaoCodigo", "Data", "TipoMedicaoChuvas", "Total"]]

    # get only TipoMedicaoChuvas == 1
    df = df[df["TipoMedicaoChuvas"] == 1]

    # cull null or zero values for Total
    df = df[df["Total"] > 0]

    # cull 2003 and earlier
    df = df[df["Data"].dt.year >= 2004]

    longest_df = max(longest_df, len(df))
    shortest_df = min(shortest_df, len(df))

    # skip if there is no data left
    if len(df) == 0:
        continue

    # get earliest and latest date
    earliest = df["Data"].min()
    latest = df["Data"].max()

    # rename columns for consistency
    df.columns = ["station", "date", "measurement_type", "monthly_total"]

    # merge this and stations using station code
    df = df.merge(stations, on="station")

    # remove measurement_type column since all are 1
    df = df.drop(columns=["measurement_type"])

    # print remaining data
    print(file, earliest.year, latest.year, len(df))

    # add to main db
    db = pd.concat([db, df])

print(f"Current db length = {len(db)}")
db.to_csv("chuvas.csv", index=False)

1036005_Chuvas.csv 2004 2024 275
1036007_Chuvas.csv 2004 2024 270
1040028_Chuvas.csv 2004 2024 147
1040036_Chuvas.csv 2005 2024 176
1041016_Chuvas.csv 2004 2024 140
1042012_Chuvas.csv 2004 2024 157
1046002_Chuvas.csv 2004 2022 136
1141014_Chuvas.csv 2004 2015 106
1142017_Chuvas.csv 2004 2024 167
1142020_Chuvas.csv 2004 2024 176
1143005_Chuvas.csv 2004 2015 90
1144027_Chuvas.csv 2004 2024 190
1145001_Chuvas.csv 2004 2024 201
1145004_Chuvas.csv 2004 2024 201
1145013_Chuvas.csv 2004 2024 198
1145020_Chuvas.csv 2004 2024 171
1242016_Chuvas.csv 2004 2024 164
1243000_Chuvas.csv 2004 2024 187
1243011_Chuvas.csv 2004 2024 192
1244011_Chuvas.csv 2004 2024 204
1244021_Chuvas.csv 2004 2024 162
1245016_Chuvas.csv 2004 2024 177
1245017_Chuvas.csv 2004 2024 157
1343025_Chuvas.csv 2004 2024 130
1344014_Chuvas.csv 2004 2024 170
1344017_Chuvas.csv 2004 2024 192
1344025_Chuvas.csv 2004 2024 164
1345003_Chuvas.csv 2004 2024 166
1442031_Chuvas.csv 2004 2024 142
1442032_Chuvas.csv 2004 2024 181
1442040_Chu

Por fim contamos quantas estações que se adequam aos critérios e quantos registros válidos cada sub-bacia tinha disponível.

In [16]:
print(f"Shortest df = {shortest_df}")
print(f"Longest df = {longest_df}")

stations_by_sub_basin = db.groupby("sub_basin").station.nunique()
print(f"Stations by sub-basin:\n{stations_by_sub_basin}")

rows_by_sub_basin = db.groupby("sub_basin").date.count()
print(f"Rows by sub-basin:\n{rows_by_sub_basin}")


Shortest df = 0
Longest df = 275
Stations by sub-basin:
sub_basin
40    40
41    22
42    20
43     6
44    19
45    11
46    14
47    10
48    21
49    10
Name: station, dtype: int64
Rows by sub-basin:
sub_basin
40    9079
41    4729
42    3955
43    1139
44    3421
45    1872
46    2430
47    1520
48    4032
49    2433
Name: date, dtype: int64
